# Preprocess

In [ ]:
import re

class TextPreProc():
    def __init__(self, stopwords_path = "./stopwords-fa.txt"):
        try:
            with open(stopwords_path, 'r', encoding='utf-8') as f:
                self.stopwords = set(f.read().splitlines())
        except FileNotFoundError:
            self.stopwords = set()

    def replace_persian_char(self , text):
        replacements = {'ي': 'ی', 'ك': 'ک'}
        for arab, fa in replacements.items():
            text = text.replace(arab, fa)
        return text
    
    def convert_numbers_to_persian(self , text):
        text = text.replace('0', '۰')
        text = text.replace('1', '۱')
        text = text.replace('2', '۲')
        text = text.replace('3', '۳')
        text = text.replace('4', '۴')
        text = text.replace('5', '۵')
        text = text.replace('6', '۶')
        text = text.replace('7', '۷')
        text = text.replace('8', '۸')
        text = text.replace('9', '۹')
        return text
    
    def remove_links_and_tags(self , text):
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'#\w+', '', text)
        text = re.sub(r'RT', '', text)
        text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
        return text

    def remove_whitespaces(self , text):
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def remove_persian_stopwords(self , text):
        if self.stopwords:
            text = ' '.join([word for word in text.split() if word not in self.stopwords])
        return text

    def process_text(self , text):
        text = self.replace_persian_char(text)
        text = self.convert_numbers_to_persian(text)
        text = self.remove_links_and_tags(text)
        text = self.remove_whitespaces(text)
        text = self.remove_persian_stopwords(text)
        return text
    

In [ ]:
text = "غذااااش اصلا خوب نبوددد 😡 ... پیکش سریع بود @snappfood #بد ... 1000 تومن! :))))"
process = TextPreProc()

print(process.process_text(text))

# Embedding

## load fasttext model

In [ ]:
import fasttext
model = fasttext.load_model("cc.fa.300.bin")

print(model.get_dimension())  
print(model.get_word_vector("غذا").shape)  


## Determining sentence length‍‍

In [ ]:
import pandas as pd
import numpy as np

df_train = pd.read_csv(
    "./snappfood/train.csv",
    sep="\t"
)

sentence_lengths = df_train["comment"].apply(
    lambda x: len(x.split())
)

print(sentence_lengths.describe())


In [ ]:
for p in [90, 95, 99]:
    print(f"{p}th percentile:", np.percentile(sentence_lengths, p))


In [ ]:
MAX_LEN = 50
FAST_TEXT_DIM = 300

## Embedding operation

In [ ]:
class Embedder():
    def __init__(self, model, train_dataset: pd.DataFrame, max_length: int = MAX_LEN, dim: int = FAST_TEXT_DIM , stopwords_path = "./stopwords-fa.txt"):
        self.model = model
        self.model = model
        self.train_dataset = train_dataset
        self.max_length = max_length
        self.dim = dim
        self.preprocessor = TextPreProc(stopwords_path)


    def embed_sentence(self, text: str) -> np.ndarray:
        text = self.preprocessor.process_text(text)
        words = text.split()

        embeddings = []

        # truncation
        for w in words[:self.max_length]:
            embeddings.append(self.model.get_word_vector(w))
        
        #padding
        while len(embeddings) < self.max_length:
            embeddings.append(np.zeros(self.dim))
        
        # transpose matrix needed for Conv1d
        return np.array(embeddings).T

    def process_dataset(self):

        embedded_sentences = []

        for text in self.train_dataset["comment"]:
            emb = self.embed_sentence(text)
            embedded_sentences.append(emb)
        
        return np.stack(embedded_sentences)


In [ ]:
embedder = Embedder(model, df_train)

# Dataset and DataLoader

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

class SentimentDataset(Dataset):
    def __init__(self, dataframe, embedder):
        self.dataframe = dataframe
        self.embedder = embedder

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row['comment']
        label = row['label_id']

        input_numpy = self.embedder.embed_sentence(text)
        input_tensor = torch.tensor(input_numpy, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return input_tensor, label_tensor    

In [ ]:
train_dataset = SentimentDataset(dataframe=df_train, embedder=embedder)

df_dev = pd.read_csv(
    "./snappfood/dev.csv",
    sep="\t"
)
dev_dataset = SentimentDataset(dataframe=df_dev, embedder=embedder)

# CNN classifier

In [ ]:
import torch.nn as nn

class CNNClassifier(torch.nn.Module):
    def __init__(self, input_dim = FAST_TEXT_DIM, hidden_dim = 128 , dropout_prob=0.5):
        super(CNNClassifier, self).__init__()
        linier_dim = int((MAX_LEN / 2) * hidden_dim)
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Linear(linier_dim , 2)

    def forward(self , x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [ ]:
classifier_model = CNNClassifier()

# training

In [ ]:
import torch
import copy
from tqdm import tqdm


def train_model(model, optimizer, train_loader, val_loader, loss_module, num_epochs=10, device='cpu'):
    model = model.to(device)

    best_accuracy = 0.0
    best_weights = copy.deepcopy(model.state_dict())

    print("Start training ...")

    for epoch in range(num_epochs):
        model.train() # dropout activation
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for inputs, labels in loop:
            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            inputs, labels = inputs.to(device), labels.to(device)

            ## Step 2: Run the model on the input data
            outputs = model(inputs)

            ## Step 3: Calculate the loss
            loss = loss_module(outputs, labels)

            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero.
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad()
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()

            loop.set_postfix(loss=loss.item()) #update Progress bar

        model.eval()
        total_correct_pred = 0
        total_data = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                total_correct_pred += (preds == labels).sum().item()
                total_data += labels.size(0)

            accuracy = (total_correct_pred / total_data) * 100

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_weights = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), "best_model.pth")

    print(f"\nTraining Finished. Best Validation Accuracy: {best_accuracy:.2f}%")
    model.load_state_dict(best_weights)
    return model

# Finding hyperparameters

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader

learning_rates = [1e-3, 5e-4, 1e-4]
dropout_rates  = [0.3, 0.5]

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)

for lr in learning_rates:
    for do in dropout_rates:
        model = CNNClassifier(dropout_prob=do)

        loss = torch.nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        trained_model = train_model(
            model=model,
            optimizer=optimizer,
            train_loader=train_loader,
            val_loader=dev_loader,
            loss_module=loss,
            num_epochs=10,
        )

        print(f"Done with LR={lr}, DO={do}")

print("finish")


# Model training using specified hyperparameters

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = CNNClassifier(dropout_prob=0.3)
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

trained_model = train_model(
    model=model,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=dev_loader,
    loss_module=loss,
    num_epochs=30,
)

# model.load_state_dict(torch.load("best_model.pth"))

# Evaluation and result

In [ ]:
def evaluate(model, test_loader , device='cpu'):
    model.eval()
    total_correct_pred = 0
    total_data = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total_correct_pred += (preds == labels).sum().item()
            total_data += labels.size(0)

    accuracy = (total_correct_pred / total_data) * 100
    return accuracy

In [ ]:
BATCH_SIZE = 32

df_test = pd.read_csv("./snappfood/test.csv", sep="\t")
test_dataset = SentimentDataset(dataframe=df_test, embedder=embedder)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Accuracy on test set: {evaluate(trained_model, test_loader):.2f}%")